In [1]:
import numpy as np
import pandas as pd

In [2]:
#Defensive Rankings from ProFootballReference
#(https://www.pro-football-reference.com/years/2021/opp.htm)
D_data ={}
overall_data = pd.DataFrame([])

for i in np.arange(1970, 2022):
    D_data[i] = pd.read_excel('Defense_Rankings.xlsx', str(i), header=[1])
    D_data[i]['Season'] = i
    overall_data = pd.concat([overall_data, D_data[i]])    
    Def = overall_data.copy()
    Def.rename(columns = {'Rk':'Def_Rk','G': 'Games', 'PA': 'Points_Allowed', 'Yds':'Tot_Yards', 
                          'Ply':'Tot_Plays', 'Y/P': 'Tot_Yards/Play', 'TO':'Takeaways', 
                          '1stD':'Tot_First_Downs', 'Cmp':'Pass_Completed', 'Att': 'Pass_Attempted',
                          'Yds.1': 'Pass_Yards', 'TD':'Pass_TD', 'NY/A':'Pass_Net/Attempt', 
                          '1stD.1':'Pass_First_Downs', 'Att.1': 'Rush_Attempted', 'Yds.2': 'Rush_Yards',
                          'Yds.2':'Rush_Yards', 'TD.1':'Rush_TD', 'Y/A': 'Rush_Yds/Attempt', 
                          '1stD.2':'Rush_First_Downs', 'Pen':'Penalities','Yds.3':'Penalty_Yards',
                          '1stPy':'First_Down_By_Penalty', 'Sc%': 'Scoring_Percentage', 
                          'TO%':'Turnover_Percentage', 'Tm':'Team'}, inplace=True)
    Def.dropna(subset=['Games'], inplace=True)
    Def = Def.reset_index(drop=True)
    
    
#Offensive Rankings from ProFootballReference 
#(https://www.pro-football-reference.com/years/2021/index.htm)    
O_data = {}
overall_data1 = pd.DataFrame([])

for i in np.arange(1970, 2022):
    O_data[i] = pd.read_excel('Offense_Rankings.xlsx', str(i), header=[1])
    O_data[i]['Season'] = i
    overall_data1 = pd.concat([overall_data1, O_data[i]])
    Off = overall_data1.copy()
    Off.rename(columns = {'Rk':'Off_Rk', 'G': 'Games', 'PF': 'Points_For', 'Yds':'Tot_Yards', 
                          'Ply':'Tot_Plays', 'Y/P': 'Tot_Yards/Play', 'TO':'Turnover', 
                          '1stD':'Tot_First_Downs', 'Cmp':'Pass_Completed', 'Att': 'Pass_Attempted',
                          'Yds.1': 'Pass_Yards', 'TD':'Pass_TD', 'NY/A':'Pass_Net/Attempt',
                          '1stD.1':'Pass_First_Downs', 'Att.1': 'Rush_Attempted', 'Yds.2': 'Rush_Yards',
                          'Yds.2':'Rush_Yards', 'TD.1':'Rush_TD', 'Y/A': 'Rush_Yds/Attempt',
                          '1stD.2':'Rush_First_Downs', 'Pen':'Penalities','Yds.3':'Penalty_Yards',
                          '1stPy':'First_Down_By_Penalty', 'Sc%': 'Scoring_Percentage', 
                          'TO%':'Turnover_Percentage', 'Tm':'Team'}, inplace=True)
    Off.dropna(subset=['Games'], inplace=True)
    Off = Off.reset_index(drop=True)
    

In [3]:
#Merging Offense and Defense Dataframes together
Off = Off.drop(['FL','EXP'], axis=1)
Def = Def.drop(['FL','EXP'], axis=1)

Team = pd.merge(Def, Off, how='inner', on=['Season','Team','Games'], suffixes= ('_def', '_off'))
Team.head(5)

,Def_Rk,Team,Games,Points_Allowed,Tot_Yards_def,Tot_Plays_def,Tot_Yards/Play_def,Takeaways,Tot_First_Downs_def,Pass_Completed_def,...,Rush_Attempted_off,Rush_Yards_off,Rush_TD_off,Rush_Yds/Attempt_off,Rush_First_Downs_off,Penalities_off,Penalty_Yards_off,First_Down_By_Penalty_off,Scoring_Percentage_off,Turnover_Percentage_off
0,1.0,Minnesota Vikings,14.0,143.0,2803.0,814.0,3.4,44.0,168.0,195.0,...,508.0,1634.0,16.0,3.2,98.0,60.0,631.0,16.0,NaN,NaN
1,2.0,Detroit Lions,14.0,202.0,3448.0,756.0,4.6,44.0,186.0,194.0,...,514.0,2127.0,16.0,4.1,113.0,58.0,659.0,23.0,NaN,NaN
2,3.0,Los Angeles Rams,14.0,202.0,3548.0,826.0,4.3,35.0,195.0,196.0,...,430.0,1763.0,12.0,4.1,93.0,88.0,959.0,11.0,NaN,NaN
3,4.0,Dallas Cowboys,14.0,221.0,3569.0,855.0,4.2,39.0,205.0,193.0,...,522.0,2300.0,16.0,4.4,119.0,87.0,934.0,15.0,NaN,NaN
4,5.0,St. Louis Cardinals,14.0,228.0,3869.0,894.0,4.3,35.0,242.0,183.0,...,429.0,1998.0,18.0,4.7,110.0,84.0,896.0,12.0,NaN,NaN


In [4]:
#Scraping Conference and Division information of NFL teams for 2002-2021
all_tables = pd.read_html('https://en.wikipedia.org/wiki/National_Football_League')

conf = all_tables[3]
conf = conf[['Division[66]', 'Club[66]']]
afc = conf.iloc[0:16].copy()
nfc = conf.iloc[17:33].copy()

nfc.columns = nfc.columns.droplevel()
afc.columns = afc.columns.droplevel()

afc.columns.values[1] = 'Team'
afc.columns.values[0] = 'Division'
afc['Conference'] = 'AFC'

nfc.columns.values[1] = 'Team'
nfc.columns.values[0] = 'Division'
nfc['Conference'] = 'NFC'

nfl = pd.concat([afc, nfc], axis=0)
nfl['Team'] = nfl['Team'].str.replace('*','', regex=True)
nfl['Team'] = nfl['Team'].str.replace('†','', regex=True)
nfl.head(5)

,Division,Team,Conference
0,East,Buffalo Bills,AFC
1,East,Miami Dolphins,AFC
2,East,New England Patriots,AFC
3,East,New York Jets,AFC
4,North,Baltimore Ravens,AFC


In [5]:
# Scraping Conference and division information of NFL teams for teams prior 2001
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2001_NFL_season')
oldcon = all_tables[2]

# Cleaning table to prepare for merge
oldcon = oldcon.drop(labels=[0,2,9,17, 24, 31, 38])
oldcon = oldcon[[0]]
oldcon['Division'] = ''
oldcon['Conference'] = ''
oldcon.loc[3:7,'Division'] = 'East'
oldcon.loc[3:7,'Conference'] = 'AFC'
oldcon.loc[10:15, 'Division'] = 'Central'
oldcon.loc[10:15,'Conference'] = 'AFC'
oldcon.loc[18:22,'Division'] = 'West'
oldcon.loc[18:22,'Conference'] = 'AFC'
oldcon.loc[25:29, 'Division'] = 'East'
oldcon.loc[25:29,'Conference'] = 'NFC'
oldcon.loc[32:36,'Conference'] = 'NFC'
oldcon.loc[29:43, 'Division'] = 'West'
oldcon.loc[29:43,'Conference'] = 'NFC'
oldcon.loc[32:36, 'Division'] = 'Central'
oldcon = oldcon.drop(labels=[1,8,16,23,30,37])
oldcon = oldcon.rename(columns = {oldcon.columns[0]:'Team'})

#removing chararcters
oldcon['Team'] = oldcon['Team'].str.strip('(')
for i in range(7):
    oldcon['Team'] = oldcon['Team'].str.strip(str(i))
oldcon['Team'] = oldcon['Team'].str.strip(')')

oldcon.head(5)

,Team,Division,Conference
3,New England Patriots,East,AFC
4,Miami Dolphins,East,AFC
5,New York Jets,East,AFC
6,Indianapolis Colts,East,AFC
7,Buffalo Bills,East,AFC


In [6]:
# Merging division and conference data
Team1 = Team.loc[Team['Season'] >= 2002]
Team2 = Team.loc[Team['Season'] <= 2001]


Team1 = pd.merge(Team1, nfl, on='Team', how='left')
Team2 = pd.merge(Team2, oldcon, on='Team', how='left')

In [7]:
# Filling in nas for seasons prior to 2001
Team2['Division'] = np.where(Team2['Team'] == 'Miami Dolphins', 'East', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Miami Dolphins', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Oakland Raiders', 'West', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Oakland Raiders', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Chicago Bears', 'Central', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Chicago Bears', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'New York Jets', 'East', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'New York Jets', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'San Francisco 49ers', 'West', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'San Francisco 49ers', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Tampa Bay Buccaneers', 'Central', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Tampa Bay Buccaneers', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'St. Louis Rams', 'West', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'St. Louis Rams', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'New England Patriots', 'East', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'New England Patriots', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Green Bay Packers', 'Central', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Green Bay Packers', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Pittsburgh Steelers', 'Central', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Pittsburgh Steelers', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Baltimore Ravens', 'Central', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Baltimore Ravens', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Philadelphia Eagles', 'East', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Philadelphia Eagles', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Tennessee Oilers', 'Central', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Tennessee Oilers', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Houston Oilers', 'Central', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Houston Oilers', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Los Angeles Rams', 'West', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Los Angeles Rams', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Los Angeles Raiders', 'West', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Los Angeles Raiders', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Phoenix Cardinals', 'East', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Phoenix Cardinals', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'St. Louis Cardinals', 'East', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'St. Louis Cardinals', 'NFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Baltimore Colts', 'East', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Baltimore Colts', 'AFC', Team2['Conference'])

Team2['Division'] = np.where(Team2['Team'] == 'Boston Patriots', 'East', Team2['Division'])
Team2['Conference'] = np.where(Team2['Team'] == 'Boston Patriots', 'AFC', Team2['Conference'])

In [8]:
# Filling in nas after conference realignment

Team1['Division'] = np.where(Team1['Team'] == 'Oakland Raiders', 'West', Team1['Division'])
Team1['Conference'] = np.where(Team1['Team'] == 'Oakland Raiders', 'AFC', Team1['Conference'])

Team1['Division'] = np.where(Team1['Team'] == 'Washington Redskins', 'East', Team1['Division'])
Team1['Conference'] = np.where(Team1['Team'] == 'Washington Redskins', 'NFC', Team1['Conference'])

Team1['Division'] = np.where(Team1['Team'] == 'Washington Football Team', 'East', Team1['Division'])
Team1['Conference'] = np.where(Team1['Team'] == 'Washington Football Team', 'NFC', Team1['Conference'])

Team1['Division'] = np.where(Team1['Team'] == 'San Diego Chargers', 'West', Team1['Division'])
Team1['Conference'] = np.where(Team1['Team'] == 'San Diego Chargers', 'AFC', Team1['Conference'])

Team1['Division'] = np.where(Team1['Team'] == 'St. Louis Rams', 'West', Team1['Division'])
Team1['Conference'] = np.where(Team1['Team'] == 'St. Louis Rams', 'NFC', Team1['Conference'])

In [9]:
# combing post and pre conference realignment divisions
Team = pd.concat([Team1, Team2], axis=0)
Team

,Def_Rk,Team,Games,Points_Allowed,Tot_Yards_def,Tot_Plays_def,Tot_Yards/Play_def,Takeaways,Tot_First_Downs_def,Pass_Completed_def,...,Rush_TD_off,Rush_Yds/Attempt_off,Rush_First_Downs_off,Penalities_off,Penalty_Yards_off,First_Down_By_Penalty_off,Scoring_Percentage_off,Turnover_Percentage_off,Division,Conference
0,1.0,Tampa Bay Buccaneers,16.0,196.0,4044.0,963.0,4.2,38.0,236.0,259.0,...,6.0,3.8,90.0,103.0,789.0,25.0,32.6,10.7,South,NFC
1,2.0,Philadelphia Eagles,16.0,241.0,4754.0,1009.0,4.7,38.0,272.0,333.0,...,15.0,4.5,124.0,111.0,897.0,23.0,36.0,12.0,East,NFC
2,3.0,New York Giants,16.0,279.0,4949.0,985.0,5.0,25.0,266.0,311.0,...,14.0,3.9,95.0,113.0,955.0,33.0,31.2,13.8,East,NFC
3,4.0,Miami Dolphins,16.0,301.0,4656.0,974.0,4.8,30.0,285.0,294.0,...,24.0,4.7,139.0,103.0,859.0,24.0,34.6,15.7,East,AFC
4,5.0,Carolina Panthers,16.0,302.0,4646.0,1022.0,4.5,33.0,271.0,293.0,...,11.0,3.5,82.0,94.0,785.0,23.0,19.4,17.5,South,NFC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,27.0,New Orleans Saints,16.0,409.0,5070.0,948.0,5.3,30.0,284.0,278.0,...,8.0,4.1,87.0,119.0,1025.0,23.0,31.6,16.8,West,NFC
897,28.0,Carolina Panthers,16.0,410.0,5943.0,1057.0,5.6,36.0,304.0,306.0,...,9.0,3.9,68.0,87.0,747.0,24.0,22.4,15.9,West,NFC
898,29.0,Buffalo Bills,16.0,420.0,5292.0,969.0,5.5,19.0,302.0,284.0,...,9.0,4.2,75.0,123.0,954.0,32.0,24.5,16.8,East,AFC
899,30.0,Detroit Lions,16.0,424.0,5521.0,1015.0,5.4,22.0,321.0,312.0,...,8.0,4.0,74.0,116.0,1081.0,31.0,25.0,19.7,Central,NFC


In [10]:
# Scraping the teams and results of NFL playoffs since 1970 (wilcard was adapted in 1978) 
playoffs = pd.read_html('https://en.wikipedia.org/wiki/NFL_playoff_results')
wildcard = playoffs[3]
divisional = playoffs[4]
conference = playoffs[5]

In [11]:
# Cleaning Wildcard results
wildcard = playoffs[3]
wildcard.columns = wildcard.columns.droplevel()
wildcard['Year'] = wildcard['Year'].astype(str).str[:-3]
wildcard = wildcard.drop(columns=['Ref','Score'])
wildcard = wildcard[wildcard['Year'] != 'Year']
wildcard = wildcard[wildcard['Year'] != 'Y']

# Split and concat
new = wildcard.iloc[:,[0,3,4]]
wildcard = wildcard.iloc[:,0:3]
wildcard = pd.concat([new,wildcard], ignore_index=True)

# Split concat to indicate wildcard teams and merge to final team dataframe
wildcard1 = wildcard[['Year', 'Winner']]
wildcard2 = wildcard[['Year', 'Loser']]
wildcard1 = wildcard1.rename(columns={'Winner':'Team','Year':'Season'})
wildcard2 = wildcard2.rename(columns={'Loser':'Team', 'Year':'Season'})
wildcard = pd.concat([wildcard1, wildcard2], ignore_index=True)

# Adding column to indicate wildcard team 
wildcard['wildcard'] = 1

# Merging to Team df
wildcard['Season'] = wildcard['Season'].astype(int) 
Team =  pd.merge(Team, wildcard, how='left', on=['Season','Team'])

Team.head(5)

,Def_Rk,Team,Games,Points_Allowed,Tot_Yards_def,Tot_Plays_def,Tot_Yards/Play_def,Takeaways,Tot_First_Downs_def,Pass_Completed_def,...,Rush_Yds/Attempt_off,Rush_First_Downs_off,Penalities_off,Penalty_Yards_off,First_Down_By_Penalty_off,Scoring_Percentage_off,Turnover_Percentage_off,Division,Conference,wildcard
0,1.0,Tampa Bay Buccaneers,16.0,196.0,4044.0,963.0,4.2,38.0,236.0,259.0,...,3.8,90.0,103.0,789.0,25.0,32.6,10.7,South,NFC,NaN
1,2.0,Philadelphia Eagles,16.0,241.0,4754.0,1009.0,4.7,38.0,272.0,333.0,...,4.5,124.0,111.0,897.0,23.0,36.0,12.0,East,NFC,NaN
2,3.0,New York Giants,16.0,279.0,4949.0,985.0,5.0,25.0,266.0,311.0,...,3.9,95.0,113.0,955.0,33.0,31.2,13.8,East,NFC,1.0
3,4.0,Miami Dolphins,16.0,301.0,4656.0,974.0,4.8,30.0,285.0,294.0,...,4.7,139.0,103.0,859.0,24.0,34.6,15.7,East,AFC,NaN
4,5.0,Carolina Panthers,16.0,302.0,4646.0,1022.0,4.5,33.0,271.0,293.0,...,3.5,82.0,94.0,785.0,23.0,19.4,17.5,South,NFC,NaN


In [12]:
Team.shape

(1541, 54)

In [13]:
# cleaning Conference championship results 
conference.columns = conference.columns.droplevel()
conference['Year'] = conference['Year'].astype(str).str[:-3]
conference = conference.drop(columns=['Ref.','Score'])
conference = conference[conference['Year'] != 'Y']

# split and concat
new = conference.iloc[:,[0,3,4]]
conference = conference.iloc[:,0:3]
conference = pd.concat([new, conference], ignore_index=True)

# split and concat to indicate conference championship teams and merge to final team dataframe
conference1 = conference[['Year', 'Winner']]
conference2 = conference[['Year', 'Loser']]
conference1 = conference1.rename(columns={'Winner':'Team','Year':'Season'})
conference2 = conference2.rename(columns={'Loser':'Team', 'Year':'Season'})
conference = pd.concat([conference1, conference2], ignore_index=True)

# adding column to indicate wildcard team 
conference['conf_championship'] = 1

# merging to Team df
conference['Season'] = conference['Season'].astype(int) 
Team =  pd.merge(Team, conference, how='left', on=['Season','Team'])

Team.head(5)

,Def_Rk,Team,Games,Points_Allowed,Tot_Yards_def,Tot_Plays_def,Tot_Yards/Play_def,Takeaways,Tot_First_Downs_def,Pass_Completed_def,...,Rush_First_Downs_off,Penalities_off,Penalty_Yards_off,First_Down_By_Penalty_off,Scoring_Percentage_off,Turnover_Percentage_off,Division,Conference,wildcard,conf_championship
0,1.0,Tampa Bay Buccaneers,16.0,196.0,4044.0,963.0,4.2,38.0,236.0,259.0,...,90.0,103.0,789.0,25.0,32.6,10.7,South,NFC,NaN,1.0
1,2.0,Philadelphia Eagles,16.0,241.0,4754.0,1009.0,4.7,38.0,272.0,333.0,...,124.0,111.0,897.0,23.0,36.0,12.0,East,NFC,NaN,1.0
2,3.0,New York Giants,16.0,279.0,4949.0,985.0,5.0,25.0,266.0,311.0,...,95.0,113.0,955.0,33.0,31.2,13.8,East,NFC,1.0,NaN
3,4.0,Miami Dolphins,16.0,301.0,4656.0,974.0,4.8,30.0,285.0,294.0,...,139.0,103.0,859.0,24.0,34.6,15.7,East,AFC,NaN,NaN
4,5.0,Carolina Panthers,16.0,302.0,4646.0,1022.0,4.5,33.0,271.0,293.0,...,82.0,94.0,785.0,23.0,19.4,17.5,South,NFC,NaN,NaN


In [14]:
playoffs = pd.read_html('https://en.wikipedia.org/wiki/NFL_playoff_results')
divisional = playoffs[4]

# Cleaning divisional playoff results
divisional.columns = divisional.columns.droplevel()
divisional['Year'] = divisional['Year'].astype(str).str[:-3]
divisional = divisional.drop(columns=['Ref','Score'], axis=1)
divisional = divisional[divisional['Year'] != 'Y']

# split and concat
new = divisional.iloc[:,[0,3,4]]
divisional = divisional.iloc[:,0:3]
divisional = pd.concat([new, divisional], ignore_index=True)

# split and concat to indicate divisional playoff teams and merge to final team dataframe
divisional1 = divisional[['Year', 'Winner']]
divisional2 = divisional[['Year', 'Loser']]
divisional1 = divisional1.rename(columns={'Winner':'Team','Year':'Season'})
divisional2 = divisional2.rename(columns={'Loser':'Team', 'Year':'Season'})
divisional = pd.concat([divisional1, divisional2], ignore_index=True)

# adding column to indicate divisional team 
divisional['divisional_game'] = 1

# merging to Team df
divisional['Season'] = divisional['Season'].astype(int) 
Team =  pd.merge(Team, divisional, how='left', on=['Season','Team'])

Team.head(5)

,Def_Rk,Team,Games,Points_Allowed,Tot_Yards_def,Tot_Plays_def,Tot_Yards/Play_def,Takeaways,Tot_First_Downs_def,Pass_Completed_def,...,Penalities_off,Penalty_Yards_off,First_Down_By_Penalty_off,Scoring_Percentage_off,Turnover_Percentage_off,Division,Conference,wildcard,conf_championship,divisional_game
0,1.0,Tampa Bay Buccaneers,16.0,196.0,4044.0,963.0,4.2,38.0,236.0,259.0,...,103.0,789.0,25.0,32.6,10.7,South,NFC,NaN,1.0,1.0
1,2.0,Philadelphia Eagles,16.0,241.0,4754.0,1009.0,4.7,38.0,272.0,333.0,...,111.0,897.0,23.0,36.0,12.0,East,NFC,NaN,1.0,1.0
2,3.0,New York Giants,16.0,279.0,4949.0,985.0,5.0,25.0,266.0,311.0,...,113.0,955.0,33.0,31.2,13.8,East,NFC,1.0,NaN,NaN
3,4.0,Miami Dolphins,16.0,301.0,4656.0,974.0,4.8,30.0,285.0,294.0,...,103.0,859.0,24.0,34.6,15.7,East,AFC,NaN,NaN,NaN
4,5.0,Carolina Panthers,16.0,302.0,4646.0,1022.0,4.5,33.0,271.0,293.0,...,94.0,785.0,23.0,19.4,17.5,South,NFC,NaN,NaN,NaN


In [15]:
SB = pd.read_csv('SuperBowlQBs.csv')

# removing pre-1970 SB (before 1970 NFL merger)
SB.drop(SB.loc[0:3].index, inplace=True)
SB.drop(columns=['Super Bowl'], inplace=True)
SB.rename(columns={'Team.1':'Losing_Team', 'Winning QB': 'SBWin_QB', 'Team':'Win_Team',
                   'Losing QB': 'SBLosing_QB'}, inplace=True)

# removing miscellaneous characters
SB['Win_Team'] = SB['Win_Team'].map(lambda x: str(x)[:-1])
SB['Losing_Team'] = SB['Losing_Team'].map(lambda x: str(x)[:-1])

# split and concat
SB1 = SB[['Season', 'Win_Team']]
SB2 = SB[['Season', 'Losing_Team']]
SB1 = SB1.rename(columns={'Win_Team':'Team'})
SB2 = SB2.rename(columns= {'Losing_Team': 'Team'})
SB = pd.concat([SB1, SB2], ignore_index=True)

# adding column to indicate superbowl team
SB['super_bowl'] = 1

# merging to Team df
SB['Season'] = SB['Season'].astype(int) 
Team =  pd.merge(Team, SB, how='left', on=['Season','Team'])

In [16]:
Team.shape

(1541, 57)

In [17]:
Team

,Def_Rk,Team,Games,Points_Allowed,Tot_Yards_def,Tot_Plays_def,Tot_Yards/Play_def,Takeaways,Tot_First_Downs_def,Pass_Completed_def,...,Penalty_Yards_off,First_Down_By_Penalty_off,Scoring_Percentage_off,Turnover_Percentage_off,Division,Conference,wildcard,conf_championship,divisional_game,super_bowl
0,1.0,Tampa Bay Buccaneers,16.0,196.0,4044.0,963.0,4.2,38.0,236.0,259.0,...,789.0,25.0,32.6,10.7,South,NFC,NaN,1.0,1.0,1.0
1,2.0,Philadelphia Eagles,16.0,241.0,4754.0,1009.0,4.7,38.0,272.0,333.0,...,897.0,23.0,36.0,12.0,East,NFC,NaN,1.0,1.0,NaN
2,3.0,New York Giants,16.0,279.0,4949.0,985.0,5.0,25.0,266.0,311.0,...,955.0,33.0,31.2,13.8,East,NFC,1.0,NaN,NaN,NaN
3,4.0,Miami Dolphins,16.0,301.0,4656.0,974.0,4.8,30.0,285.0,294.0,...,859.0,24.0,34.6,15.7,East,AFC,NaN,NaN,NaN,NaN
4,5.0,Carolina Panthers,16.0,302.0,4646.0,1022.0,4.5,33.0,271.0,293.0,...,785.0,23.0,19.4,17.5,South,NFC,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,27.0,New Orleans Saints,16.0,409.0,5070.0,948.0,5.3,30.0,284.0,278.0,...,1025.0,23.0,31.6,16.8,West,NFC,NaN,NaN,NaN,NaN
1537,28.0,Carolina Panthers,16.0,410.0,5943.0,1057.0,5.6,36.0,304.0,306.0,...,747.0,24.0,22.4,15.9,West,NFC,NaN,NaN,NaN,NaN
1538,29.0,Buffalo Bills,16.0,420.0,5292.0,969.0,5.5,19.0,302.0,284.0,...,954.0,32.0,24.5,16.8,East,AFC,NaN,NaN,NaN,NaN
1539,30.0,Detroit Lions,16.0,424.0,5521.0,1015.0,5.4,22.0,321.0,312.0,...,1081.0,31.0,25.0,19.7,Central,NFC,NaN,NaN,NaN,NaN


In [18]:
Team.to_csv('team_analysis_final.csv')